Importing the neccessary libraries

In [1]:
import pandas as pd
import numpy as np

Upload & load dataset (Google Colab)

In [6]:
from google.colab import files
uploaded = files.upload()

Saving Housing Price.csv to Housing Price.csv


Replace with your uploaded file name and View Data

In [7]:
file_name = 'Housing Price.csv'
df = pd.read_csv(file_name)
print(df.head())
print(df.info())

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 colu

Basic preprocessing .Handle missing values

In [8]:
df = df.dropna()

Separate features and target CHANGE target column name if needed

In [9]:
target = 'price'
# <-- update if different
X = df.drop(columns=[target])
y = df[target]

Encode categorical variables

In [10]:
X = pd.get_dummies(X, drop_first=True)

Train-test split & scaling Purpose Create training and testing data. Scale features for distance based models.

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

**Regression evaluation metrics**

Purpose Measure model performance

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def regression_metrics(y_true, y_pred):
    return {
        'MAE': mean_absolute_error(y_true, y_pred),
        'MSE': mean_squared_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'R2': r2_score(y_true, y_pred)
    }

**Linear Regression**

Purpose Baseline regression model.

In [13]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_test_scaled)
print('Linear Regression:', regression_metrics(y_test, y_pred_lr))

Linear Regression: {'MAE': 970043.4039201642, 'MSE': 1754318687330.6677, 'RMSE': np.float64(1324506.96009144), 'R2': 0.6529242642153177}


**Multiple Linear Regression**

Purpose Same algorithm. Uses many features.

In [14]:
mlr = LinearRegression()
mlr.fit(X_train_scaled, y_train)

y_pred_mlr = mlr.predict(X_test_scaled)
print('Multiple Linear Regression:', regression_metrics(y_test, y_pred_mlr))


Multiple Linear Regression: {'MAE': 970043.4039201642, 'MSE': 1754318687330.6677, 'RMSE': np.float64(1324506.96009144), 'R2': 0.6529242642153177}


**Polynomial Regression**

Purpose Capture non linear relationships.

In [15]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

pr = LinearRegression()
pr.fit(X_train_poly, y_train)

y_pred_pr = pr.predict(X_test_poly)
print('Polynomial Regression:', regression_metrics(y_test, y_pred_pr))

Polynomial Regression: {'MAE': 1034749.2706758835, 'MSE': 1901686413946.4487, 'RMSE': np.float64(1379016.466162188), 'R2': 0.6237689217365155}


**KNN Regression**

Purpose
Predict based on nearest data points.

In [16]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred_knn = knn.predict(X_test_scaled)
print('KNN Regression:', regression_metrics(y_test, y_pred_knn))

KNN Regression: {'MAE': 999594.6055045872, 'MSE': 1953996997258.202, 'RMSE': np.float64(1397854.4263471079), 'R2': 0.6134197563748457}


**Decision Tree Regression**

Purpose Rule based regression model.

In [17]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)
print('Decision Tree Regression:', regression_metrics(y_test, y_pred_dt))

Decision Tree Regression: {'MAE': 1195266.0550458715, 'MSE': 2642802637614.6787, 'RMSE': np.float64(1625669.904259373), 'R2': 0.4771459275854347}


**Logistic Regression**

Purpose Convert problem to binary classification.

Logic Above median price = 1 Below median price = 0

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

median_price = y.median()
y_class = (y > median_price).astype(int)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X, y_class, test_size=0.2, random_state=42
)

X_train_c = scaler.fit_transform(X_train_c)
X_test_c = scaler.transform(X_test_c)

log_reg = LogisticRegression()
log_reg.fit(X_train_c, y_train_c)

y_pred_log = log_reg.predict(X_test_c)

print('Accuracy:', accuracy_score(y_test_c, y_pred_log))
print(confusion_matrix(y_test_c, y_pred_log))
print(classification_report(y_test_c, y_pred_log))


Accuracy: 0.8532110091743119
[[47  4]
 [12 46]]
              precision    recall  f1-score   support

           0       0.80      0.92      0.85        51
           1       0.92      0.79      0.85        58

    accuracy                           0.85       109
   macro avg       0.86      0.86      0.85       109
weighted avg       0.86      0.85      0.85       109



**Unseen data prediction**

Purpose Test model on new data.

In [19]:
unseen = X_test.iloc[:5]
unseen_scaled = scaler.transform(unseen)

print('Unseen Predictions:', lr.predict(unseen_scaled))


Unseen Predictions: [5164653.90033967 7224722.29802166 3109863.24240338 4612075.3272256
 3294646.25725956]
